# 理论知识查漏补缺

## 评价指标之拟合优度$R^2$

\begin{align*}
R^2
&=1-\frac{SSE}{SST} 
=1-\frac{\sum\limits_i{(\hat{y_i}-y_i)^2}}{\sum\limits_i{(y_i-\bar{y_i})^2}} \\
&=\frac{SSR}{SST}
=\frac{\sum\limits_i{(\hat{y_i}-\bar{y_i}^2)}}{\sum\limits_i{(y_i-\bar{y_i})^2}} \\
\end{align*}

若没有自变量，在预测$y_i$的时候会用均值$\bar{y}$代替，那么损失就为SST，也代表y的波动程度；

若用自变量来解释y，SSR由X的波动引起，表示可以被自变量解释的部分；而SSE由X之外的未知因素引起，表示不可以被自变量解释的部分。

因此**$R^2$表示因变量的波动中能用自变量解释的比例**

## 参数求解之牛顿法

牛顿法可以用来求解方程，选定一初始点$(x_0,f(x_0))$，过该点以斜率$f'(x_0)$画一条直线，得到它与x轴的交点$x_1$，如此反复迭代下去

$$x_1=x_0-\frac{f(x_0)}{f'(x_0)}$$


对于损失函数$l(w)$，我们想取其最小时对应的w，因此只要求解方程$l'(w)=0$即可。

**从泰勒展开的角度看牛顿法：**

将$l(w)$进行二阶泰勒展开：
$$l(w)=l(w_0)+l'(w_0)(w-w_0)+\frac{1}{2}l''(w_0)(w-w_0)^2$$

对上式两边关于w求导：
$$l'(w)=l'(w_0)+l''(w_0)(w-w_0)=0$$

得
$$w\gets w-\frac{f'(w)}{f''(w)}$$

当w是向量的时候，$f''(w)$的位置就变成了H，也就是$l(w)$的海森矩阵；

**牛顿法收敛速度很快，但是海森矩阵计算十分复杂**，当参数维度很大的时候会消耗大量的计算资源，因此可以使用其他矩阵代替海森矩阵，引入拟牛顿法。


# python实现线性回归参数求解

In [1]:
import numpy as np

生成数据

In [3]:
X = np.random.randn(500,4)
trueW = np.array([1,2,3,4])
y = X.dot(trueW)

训练集测试集划分

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)

## 最小二乘法

$$Y=XW$$

$$LOSS=(XW-Y)^T(XW-Y)$$

$LOSS$关于参数W求导得

$$W=(X^TX)^{-1}X^TY$$

In [117]:
class LR_OLS:
    def __init__(self):
        pass
    def fit(self,X,y):
        y = y.reshape(-1,1)
        self.w = np.linalg.inv(np.dot(X.T,X)).dot(X.T).dot(y)
        print('w_predict:',self.w)
    def predict(self,X):
        return np.dot(X,self.w)
    def score(self,y_hat,y):
        y = y.reshape(-1,1)
        SSE = sum((y_hat-y)**2)
        SST = sum((y-y.mean())**2)
        self.r2 = (1 - SSE/SST)[0]
        return round(self.r2,2)

In [118]:
lrols = LR_OLS()
lrols.fit(X_train,y_train)
y_pre = lrols.predict(X_test) 
print('score:',lrols.score(y_pre,y_test))

w_predict: [[1.]
 [2.]
 [3.]
 [4.]]
score: 1.0


## 梯度下降法

$$W\gets W-\alpha X^T(\hat{y}-y)$$

迭代终止条件：
1. 当迭代次数超过阈值以后，停止迭代
2. 当两次迭代的参数变化小于设定的阈值之后停止迭代

In [114]:
class LR_GD:
    def __init__(self):
        pass
    def fit(self,X,y,alpha = 0.2, threshold=1e-10):
        y = y.reshape(-1,1)
        #初始值设置
        self.w = np.zeros((X.shape[1],1))
        #两次迭代参数的变化值初始设置
        w_delta = 10e5
        while w_delta > threshold:
            #计算梯度
            y_hat = X.dot(self.w) 
            dw = X.T.dot(y_hat - y)/X.shape[0]
            #梯度下降
            w_new = self.w - alpha * dw
            #w变化程度
            w_delta = abs(w_new - self.w).sum()
            self.w = w_new
        print('w_predict:',self.w)       
        return             
    def predict(self,X):
        return np.dot(X,self.w)
    def score(self,y_hat,y):
        y = y.reshape(-1,1)
        SSE = sum((y_hat-y)**2)
        SST = sum((y-y.mean())**2)
        self.r2 = (1 - SSE/SST)[0]
        return round(self.r2,2)
        

In [115]:
lrgd = LR_GD()
lrgd.fit(X_train,y_train)
y_pre = lrgd.predict(X_test) 
print('score:',lrgd.score(y_pre,y_test))

w_predict: [[1.]
 [2.]
 [3.]
 [4.]]
score: 1.0


## ！！容易踩的坑

```python
y.reshape(-1,1)
```
y一定要改变一下维度，不然容易发生以下情况

In [119]:
y_ = np.ones(100)
y_.shape

(100,)

In [121]:
y__ = np.ones((100,1))
y__.shape

(100, 1)

In [122]:
(y_-y__).shape

(100, 100)